# Simularium Conversion Tutorial : CytoSim Data

In [1]:
from IPython.display import Image
import numpy as np
from simulariumio import Converter

This notebook provides example python code for converting your own simulation trajectories into the format consumed by the Simularium Viewer. It creates a .simularium JSON file which you can drag and drop onto the viewer like this:

![title](img/drag_drop.gif)

***
## Prepare your spatial data

The Simularium custom data Converter consumes spatiotemporal data from CytoSim. 

To see how to generate these files, check Cytosim documentation here: https://gitlab.com/f.nedelec/cytosim-stable/-/blob/master/doc/sim/report.md
* for Fibers, use the command `./report fiber:points > fiber_points.txt`, which will create `fiber_points.txt`
* for Solids, use the command `./report solid > solids.txt`, which will create `solids.txt`
* for Singles, use the command `./report single:position > singles.txt`, which will create `singles.txt`
* for Couples, use the command `./report couple:position > couples.txt`, which will create `couples.txt`

The Converter requires the following data:
* **box_size** : *np.ndarray (shape = [3])*
    * A numpy ndarray containing the XYZ dimensions of the simulation bounding volume
* **data** : *Dict[str, Any]*
    * **fibers** : *Dict[str, Any]*
        * **filepath** : *str*
            * path to fiber_points.txt
        * **types** : *Dict[str, str]* (optional)
            * the type index from Cytosim data mapped to display names for each type of fiber
            * Default: "fiber[type index from Cytosim data]"
    * **solids** : *Dict[str, Any]*
        * **filepath** : *str*
            * path to fiber_points.txt
        * **types** : *Dict[str, str]* (optional)
            * the type index from Cytosim data mapped to display names for each type of solid
            * Default: "solid[type index from Cytosim data]"
        * **radii** : *Dict[str, float]* (optional)
            * the type index from Cytosim data mapped to radius for each type of solid
            * Default: 1.0
    * **singles** : *Dict[str, Any]*
        * **filepath** : *str*
            * path to fiber_points.txt
        * **types** : *Dict[str, str]* (optional)
            * the type index from Cytosim data mapped to display names for each type of single
            * Default: "single[type index from Cytosim data]"
            * Default: "solid[type index from Cytosim data]"
        * **radii** : *Dict[str, float]* (optional)
            * the type index from Cytosim data mapped to radius for each type of single
            * Default: 1.0
    * **couples** : *Dict[str, Any]*
        * **filepath** : *str*
            * path to fiber_points.txt
        * **types** : *Dict[str, str]* (optional)
            * the type index from Cytosim data mapped to display names for each type of couple
            * Default: "couple[type index from Cytosim data]"
            * Default: "solid[type index from Cytosim data]"
        * **radii** : *Dict[str, float]* (optional)
            * the type index from Cytosim data mapped to radius for each type of couple
            * Default: 1.0
    
Optionally, you can also include:
* **scale_factor** : *float*
    * A multiplier for the Cytosim scene, use if visualization is too large or small
    * Default: 1.0
* **plots** : *Dict[str, Any]*
    * An object containing plot data already in Simularium format

In [2]:
box_size = 2.

example_data = {
    "box_size" : np.array([box_size, box_size, box_size]),
    "data" : {
        "fibers" : {
            "filepath" : "../simulariumio/tests/data/cytosim/aster_pull3D_actin_solid/fiber_points.txt",
            "types" : {
                "1" : "microtubule",
                "2" : "actin"
            }
        },
        "solids" : {
            "filepath" : "../simulariumio/tests/data/cytosim/aster_pull3D_actin_solid/solids.txt",
            "types" : {
                "1" : "aster",
                "2" : "vesicle"
            },
            "radii" : {
                "1" : 0.1,
                "2" : 0.1
            }
        },
        "singles" : {
            "filepath" : "../simulariumio/tests/data/cytosim/aster_pull3D_actin_solid/singles.txt",
            "types" : {
                "1" : "dynein",
                "2" : "kinesin"
            },
            "radii" : {
                "1" : 0.01,
                "2" : 0.01
            }
        }
    },
    "scale_factor" : 100.0
}

## Convert and save as .simularium JSON file

Once your data is shaped like in the `example_data` object, you can use the converter to generate the file at the given path:

In [3]:
Converter(example_data, "cytosim").write_JSON("/Users/blairl/Desktop/cytosim_example")

## Visualize in the Simularium viewer

In your browser, either Firefox or Chrome, navigate to https://staging.agentviz.allencell.org/ and drag your file onto the center viewer window. 

*For now you'll first have to choose an example trajectory and close the load window. Once the example trajectory loads, you can drop your own file in to replace it. We'll fix this soon :)*